In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types._

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)
    .withColumn("timestamp", lit(current_timestamp()))

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string, timestamp: timestamp]


[tweet: string, timestamp: timestamp]

In [4]:
val model = PipelineModel.load("/home/jovyan/models/spark-ml-model")

model = pipeline_3dfeba337f5a


pipeline_3dfeba337f5a

In [5]:
val predictionsDF = model.transform(inputDF)

predictionsDF = [tweet: string, timestamp: timestamp ... 6 more fields]


[tweet: string, timestamp: timestamp ... 6 more fields]

In [6]:
import org.apache.spark.sql.functions._
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [7]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.persist()
    
    val cleanProbability = batchDF.select(getProbability($"probability").alias("clean_probability"))
    cleanProbability.write.mode("append").format("console").save()
    
    val windowCount = batchDF
        .withWatermark("timestamp", "20 seconds")
        .groupBy(window($"timestamp", "1 minute", "1 minute"), $"prediction")
        .count()
    windowCount.write.mode("append").format("console").save()
    
    batchDF.unpersist()
}.start()

spark.streams.awaitAnyTermination()

+-------------------+
|  clean_probability|
+-------------------+
|  0.506119085226794|
|0.44409918165014944|
|0.49065237891017643|
| 0.5115763550216391|
| 0.5021846446881393|
|0.48400688875371084|
|  0.506119085226794|
| 0.5095230413582319|
| 0.4984422398503117|
| 0.5115763550216391|
| 0.5594890772238319|
| 0.5115763550216391|
|0.44409918165014944|
| 0.5115763550216391|
+-------------------+

+--------------------+----------+-----+
|              window|prediction|count|
+--------------------+----------+-----+
|[2020-01-18 00:22...|       0.0|    5|
|[2020-01-18 00:22...|       1.0|    9|
+--------------------+----------+-----+

+-------------------+
|  clean_probability|
+-------------------+
| 0.5115763550216391|
| 0.4905517026720463|
|0.49689532645289997|
| 0.4967273748932942|
|0.45120694313256016|
| 0.4822831864339756|
| 0.4040311500186379|
|  0.506119085226794|
| 0.4890505295168118|
| 0.4697388308104415|
| 0.4441067980176852|
| 0.5372391189877216|
|0.47391484143806395|
|  0.50611

Name: java.lang.InterruptedException
Message: null
StackTrace:   at java.lang.Object.wait(Native Method)
  at org.apache.spark.sql.streaming.StreamingQueryManager.awaitAnyTermination(StreamingQueryManager.scala:122)